참고: https://chatgpt.com/g/g-p-67e48ab37a348191acf69fc05b7b7a47-gimdongseob/c/67e5fe5c-40c0-800a-af84-26489b345d67

In [7]:
from tensorflow import keras

# MNIST 데이터셋 가져오기
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

# 각 샘플을 1차원 배열로 재배열
# 픽셀은 0~255 사이의 정수값을 가짐 -> 255로 나누어 0~1 사이의 값으로 정규화하는것이 보통임
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

from sklearn.model_selection import train_test_split

# train, test 데이터 분할
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

## 은닉층
- 입력층과 출력층 사이에 있는 모든 층
- 은닉층에는 활성화 함수가 표시되어있다.
## 활성화 함수
- 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다.
- 이전 실습의 소프트맥스 함수 또한 활성화 함수이다.
- 뉴런 하나하나가 최종 출력을 얼마나 내보낼지를 결정하는 함수
  - 입력값을 받아서, 출력값을 결정해주는 뇌 속 필터 같은 것
- 단순히 가중치 * 입력 + 편향만 하면 전체 모델은 선형 함수가 된다. 그렇게 되면 복잡한 문제를 풀지 못하므로 **비선형성**을 추가해주는것이 활성화 함수이다.


In [8]:
from keras.layers import Input, Dense

# 활성화 함수 지정
# 밀집층 만들기
dense1 = keras.layers.Dense(100, activation="sigmoid", input_shape=(784,))
dense2 = keras.layers.Dense(10, activation="softmax")

# dense1은 은닉층, 100개의 뉴런을 가진 밀집층이다. 활성화 함수는 sigmoid이고, 매개변수의 입력 크기를 (784,)로 지정

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 심층 신경망 만들기
- 층을 추가하여 입력 데이터에 대한 연속적인 학습 진행 가능 -> 인공 신경망의 강점
- 출력층을 가장 마지막으로 둬야한다

In [9]:
model = keras.Sequential([dense1, dense2])

In [10]:
# 모델의 요약 정보
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

- 출력 크기가 None이다.
  - 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행한다.
  - 즉 미니 배치 경사법을 사용하며, 그러므로 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응하도록 None으로 설정한다.
- 두 번째 100은 은닉층의 뉴런 개수를 100으로 두었으니 당연히 100개의 출력이 나온다.
  - 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축된다.
- 모델 파라미터의 개수
  - Dense층이며, 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 존재한다.
  - 뉴런마다 1개의 절편이 있다.

- 층을 생각해보면 첫 번째 dense에서 784->100, 두 번째 dense에서 100->10으로 학습을 한다.
  - 타겟 데이터가 총 10개이므로 마지막 출력층의 뉴런 개수는 10개이다.

### 층을 추가하는 다른 방법

In [12]:
from keras.layers import Input, Dense
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [13]:
model.summary()
# name 파라미터 지정

Model: "패션 MNIST 모델"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden (Dense)                       │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from keras.layers import Input, Dense

# model에 직접 add를 해서 층을 추가할 수 있다.
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# 모델 훈련을 해보자
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7516 - loss: 0.7701
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8455 - loss: 0.4267
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8626 - loss: 0.3809
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8725 - loss: 0.3565
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8792 - loss: 0.3349


- 시그모이드 함수는 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있어 올바른 출력을 만드는데 신속하게 대응하기 어렵다.

## 렐루 함수
- f(x) = max(0, x)
- 입력이 양수이면 그대로 출력, 입력이 0보다 작으면 0 출력
- 간단하지만 강력하다

### Flatten 층
- input_shape를 지정해주면 배치 차원 제외 입력 차원을 모두 일렬로 펼치는 역할을 해준다.
- 우리가 reshape() 메서드로 1차원 배열로 펼친걸 자동으로 해주는것

In [20]:
from keras.layers import Input, Dense

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [21]:
model.summary()
# param이 0인 flatten함수 존재

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# 훈련 데이터를 다시 준비해 모델을 훈련시켜보자
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

# 0~1사이의 값으로 변환하기 위해 255를 나눠주었다.
# 훈련할 때 필요하기에 검증 데이터는 나누지 않음
# 여기선 flatten층을 사용하기 위해 reshape함수를 사용하지 않음
train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [25]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9004 - loss: 0.2790
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9041 - loss: 0.2688
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9099 - loss: 0.2609
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9085 - loss: 0.2613
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9118 - loss: 0.2516


In [24]:
# 검증 데이터로 확인
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8747 - loss: 0.3991


[0.40297526121139526, 0.871999979019165]

## 옵티마이저
- 은닉층의 뉴런 개수, 활성화 함수, 층의 종류가 다 하이퍼파라미터이다
- 케라스는 기본적으로 미니배치 경사 하강법을 사용하며 미니 배치 개수는 32개이다. fit() 의 batch_size 매개변수를 통해 이를 조정할 수 있으며, 이 또한 하이퍼파라미터이다.
- epochs 매개변수 또한 하이퍼파라미터이다.

- 케라스는 기본적으로 경사 하강법 알고리즘인 RMSprop을 사용하며, **다양한 종류의 경사 하강 알고리즘**을 제공하는데, 이를 **옵티마이저**라고 한다.

## 옵티마이저란?
- 신경망이 학습할 때, 가중치를 "어떻게" 바꿀지 결정하는 알고리즘
- 즉 **손실 함수의 값을 최소로 만드는 가중치 조합을 찾아가는 안내자** 역할이다.




## 모멘텀 최적화
- 이전 기울기의 방향성을 관성처럼 이어받아서 더 빠르게 내려가는 방식
- SGD는 그냥 경사만 따라 내려가는 느낌
- 여기에 속도가 붙으면 더 빨리 내려감 -> 이것이 모멘텀
- **momentum** 매개변수를 이용하며, 보통 0.9 이상을 지정한다.

## 네스테로프 모멘텀 최적화
- 모멘텀보다 조금 더 똑똑한 방식. 미리 한 발 앞을 보고 기울기를 계산하는 전략
- 더 예측 기반의 움직임이다.
- **nesterov** 매개변수를 이용하며, 기본값 False에서 True값으로 변경
- 대부분 기본 확률적 경사 하강법(SGD)보다 더 나은 성능을 제공한다.

## 적응적 학습률
- 학습 중에 각 가중치마다, 혹은 학습 시점마다 **학습률(learning rate)을 자동으로 조절**
- 대표적으로 Adagrad와 RMSprop, Adam이 있다.
- **compile() 매서드의 optimizer** 매개변수에서 지정할 수 있다.

In [26]:
# Adam 클래스의 매개변수 기본값을 사용해 MNIST 모델 훈련
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# compile의 매개변수 optimizer에 'adam' 지정
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7694 - loss: 0.6716
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8554 - loss: 0.4057
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8643 - loss: 0.3642
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8846 - loss: 0.3216
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8881 - loss: 0.3057


In [27]:
model.evaluate(val_scaled, val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8710 - loss: 0.3488


[0.35337570309638977, 0.8696666955947876]